In [ ]:
!pip install langchain
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install html2text
!pip install langchain-community
!pip install newspaper3k
!pip install chromadb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

import os
from langchain.vectorstores.chroma import Chroma
from langchain.text_splitter import CharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import NewsURLLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [5]:
with open(r"/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/CNN_Links.txt") as f:
    urls = [link.replace("\n", "") for link in set(f.readlines())]

loader = NewsURLLoader(urls=urls)
data = loader.load()
print("Primeira notícia: ", data[0])
print("\nSegunda notícia: ", data[1])


Primeira notícia:  page_content="CNN —\n\nColombia says it will break diplomatic relations with Israel on Thursday over its actions in Gaza.\n\nColombian President Gustavo Petro made the announcement at a rally in Bogotá’s Bolívar Square on Wednesday, describing the Israeli government’s handling of the war in Gaza as “genocidal.”\n\nIsrael launched its assault in the Palestinian territory following terror group Hamas’ attacks on October 7, which left more than 1,200 people dead and saw more than 250 taken hostage – many of whom remain in captivity today.\n\nNow nearing its eight month, Israel’s war in the isolated enclave has killed more than 34,000 people, according to the Palestinian health ministry.\n\nIsrael’s foreign minister, Israel Katz, condemned Colombia’s announcement and accused Petro of rewarding Hamas, which controls Gaza, saying he was siding with the “most despicable monsters known to humanity.” Katz also called Petro a “hate-filled, antisemitic president,” but said rela

In [6]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [7]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

#vectordb = Chroma.from_texts([], embedding=embedding, persist_directory=persist_directory)
contents = [item.page_content for item in data]
sources = [{'source_info': item.metadata['link']} for item in data]

# Initialize Chroma without specifying texts initially
vectordb = Chroma.from_texts(contents, embedding, metadatas = sources, persist_directory=persist_directory)

vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [8]:
#Declaração do modelo de embedding
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
persist_directory = '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/db'
embedding = instructor_embeddings

#Criação do objeto de gestão da vector db
vectordb = Chroma(embedding_function=embedding, persist_directory=persist_directory)

retriever = vectordb.as_retriever()
query = input("The first question:  ")
#Retorna os 3 primeiros itens relevantes para a pergunta do usuário
context = retriever.get_relevant_documents(query)[:1]

load INSTRUCTOR_Transformer
max_seq_length  512
The first question:  where in brazil there was a recent flood?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [9]:
context

[Document(page_content='CNN —\n\nAt least 57 people have been killed and a further 373 are missing as heavy rain and flooding hit the Brazilian state of Rio Grande do Sul this week.\n\nAt least 74 others have been injured amid the series of catastrophic floods that has affected 281 municipalities, according to the latest figures shared by the civil defense Saturday.\n\nThe local government has declared a state of calamity in areas where more than 67,000 people are impacted. Almost 10,000 have been displaced and more than 4,500 are in temporary shelters, the civil defense said.\n\nAuthorities are closely monitoring dams that are not designed to handle such a high volume of water but said there is no imminent risk of failure.\n\nView of a flooded house at the Sarandi neighborhood in Porto Alegre, Rio Grande do Sul state, Brazil on May 3, 2024. Anselmo Cunha/AFP/Getty Images\n\nBrazilian President Luiz Inácio Lula da Silva met with local officials overseeing the relief efforts Thursday.\n